## Data Acquisition

In [ ]:
!pip install geemap google-cloud-storage rasterio matplotlib > /dev/null 2>&1

In [53]:
import ee
import geemap
import os
from google.colab import drive
import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive
import rasterio


drive.mount('/content/drive')

# Çalışma dizinini ayarla (Google Drive'da bir klasör aç)
work_dir = '/content/drive/MyDrive/UHI-Detection-Analysis/data/raw/'
os.makedirs(work_dir, exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
#GEE
ee.Authenticate()
ee.Initialize(project='manifest-pride-258211')

**Download image from GEE and extract LST**

In [55]:
hamburg = ee.Geometry.Point(9.99, 53.55)

# Landsat 8 koleksiyonunu filtrele (2015 ve 2024 yaz ayları ilk asama olarak sadece basitce 2 yildan bir gunluk veri secildi)
def get_landsat_data(year):
    collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
        .filterBounds(hamburg) \
        .filterDate(f'{year}-06-01', f'{year}-08-31') \
        .filter(ee.Filter.lt('CLOUD_COVER', 10)) \
        .sort('CLOUD_COVER') \
        .first()
    return collection

# LST hesaplama fonksiyonu (basitleştirilmiş)
def calculate_lst(image):
    lst = image.expression(
        '(TIRS1 * 0.00341802 + 149.0) - 273.15',  # Kelvin'den Celsius'a
        {'TIRS1': image.select('ST_B10')}
    ).rename('LST')
    return lst

lst_2015 = calculate_lst(get_landsat_data(2015))
lst_2024 = calculate_lst(get_landsat_data(2024))

**Quick Visualization to Check the Data**

In [56]:
# Interactive harita oluştur
Map = geemap.Map(center=[53.55, 9.99], zoom=12)

# LST görüntülerini haritaya ekle (renk paleti: 'inferno' veya 'coolwarm')
vis_params = {
    'min': 20,  # Min LST (°C)
    'max': 40,  # Max LST (°C)
    'palette': ['blue', 'green', 'yellow', 'red']  # Veya 'inferno'
}

Map.addLayer(lst_2015, vis_params, 'LST 2015')
Map.addLayer(lst_2024, vis_params, 'LST 2024')

# Layer kontrol paneli ekle
Map.addLayerControl()
Map

Map(center=[53.55, 9.99], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

**Export to Drive**

In [61]:
import shutil

# GeoTIFF olarak export etme fonksiyonu
def export_to_drive(image, name, folder):
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=name,
        folder=folder,
        fileNamePrefix=name,
        scale=30,  # Landsat çözünürlüğü (30m)
        region=hamburg.buffer(5000).bounds(),  # Hamburg etrafında 5km buffer
        fileFormat='GeoTIFF'
    )
    task.start()
    return task

folder = 'UHI-Detection-Analysis'

# Drive'a kaydet
export_to_drive(lst_2015, 'LST_2015_Hamburg_test', folder)
export_to_drive(lst_2024, 'LST_2025_Hamburg_test', folder)

# Task'lerin tamamlanmasını bekle (Not: Bu birkaç dakika sürebilir)
import time
while True:
    tasks = ee.batch.Task.list()
    if all(task.status()['state'] in ('COMPLETED', 'FAILED') for task in tasks):
        break
    time.sleep(10)
print("Export işlemleri tamamlandı!")

Export işlemleri tamamlandı!


In [62]:
# Dosyaları taşı
src_dir = '/content/drive/MyDrive/UHI-Detection-Analysis'
dst_dir = '/content/drive/MyDrive/UHI-Detection-Analysis/data/raw'

# Hedef klasör yoksa oluştur
os.makedirs(dst_dir, exist_ok=True)

# Klasör içeriğini kontrol et
print("Kaynak klasör içeriği:", os.listdir(src_dir))

# Dosyaları taşı
for fname in os.listdir(src_dir):
    if fname.endswith('.tif') or fname.endswith('.tif.xml'):
        shutil.move(os.path.join(src_dir, fname), os.path.join(dst_dir, fname))
        print(fname, "başarıyla taşındı.")

Kaynak klasör içeriği: ['Papers', 'Meeting Notes', 'Other Topics', 'data', 'Proposals', 'LST_2025_Hamburg_test.tif', 'LST_2015_Hamburg_test.tif']
LST_2025_Hamburg_test.tif başarıyla taşındı.
LST_2015_Hamburg_test.tif başarıyla taşındı.
